In [283]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [186]:
path = 'MovieSummaries/'
character=pd.read_csv(path+'character.metadata.tsv',sep='\t',header=None, names=['wiki_movie_ID','Freebase_movie_ID','release_date','character_name','actor_birth','actor_gender','actor_height','actor_ethnicity','actor_name','actor_age','character_actor_map_ID','Freebase_character_ID','Freebase_actor_ID'])
movie = pd.read_csv(path+'movie.metadata.tsv',sep='\t',header=None,names=['wiki_movie_ID','Freebase_movie_ID','movie_name','date','revenue','runtime','languages','countries','genres'])
name = pd.read_csv(path+'name.clusters.txt',sep='\t',header=None,names=['name','character_actor_map_ID'])
summary = pd.read_csv(path+'plot_summaries.txt',sep='\t',header=None,names=['wiki_movie_ID','summary'])
tv_tropes = pd.read_csv(path+'tvtropes.clusters.txt',sep='\t',header=None,names=['name','character_actor_map'])
imdb_title = pd.read_csv(path+'IMDB_title.tsv',sep='\t')
rating = pd.read_csv(path+'ratings.tsv',sep='\t')

display(character.head())
display(movie.head())
display(name.head())
display(summary.head())
display(tv_tropes.head())
display(imdb_title.head())
display(rating.head())
print(movie.shape)
print(imdb_title.shape)
print(movie['movie_name'].unique().shape)

/Users/silviaromanato/opt/anaconda3/envs/tf1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,wiki_movie_ID,Freebase_movie_ID,release_date,character_name,actor_birth,actor_gender,actor_height,actor_ethnicity,actor_name,actor_age,character_actor_map_ID,Freebase_character_ID,Freebase_actor_ID
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg


,wiki_movie_ID,Freebase_movie_ID,movie_name,date,revenue,runtime,languages,countries,genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


,name,character_actor_map_ID
0,Stuart Little,/m/0k3w9c
1,Stuart Little,/m/0k3wcx
2,Stuart Little,/m/0k3wbn
3,John Doe,/m/0jyg35
4,John Doe,/m/0k2_zn


,wiki_movie_ID,summary
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...


,name,character_actor_map
0,absent_minded_professor,"{""char"": ""Professor Philip Brainard"", ""movie"":..."
1,absent_minded_professor,"{""char"": ""Professor Keenbean"", ""movie"": ""Richi..."
2,absent_minded_professor,"{""char"": ""Dr. Reinhardt Lane"", ""movie"": ""The S..."
3,absent_minded_professor,"{""char"": ""Dr. Harold Medford"", ""movie"": ""Them!..."
4,absent_minded_professor,"{""char"": ""Daniel Jackson"", ""movie"": ""Stargate""..."


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


,tconst,averageRating,numVotes
0,tt0000001,5.7,1922
1,tt0000002,5.8,259
2,tt0000003,6.5,1734
3,tt0000004,5.6,174
4,tt0000005,6.2,2546


(81741, 9)
(9365430, 9)
(75478,)


In [208]:
### Merge movies from IMDB and from CMU Movie Summary Corpus and from summary and rating
movie_info = movie.merge(imdb_title,left_on='movie_name',right_on='originalTitle')
movie_info = movie_info.merge(summary,on = 'wiki_movie_ID')
movie_info = movie_info.merge(rating, on = 'tconst')
movie_info = movie_info.merge(character, on = 'wiki_movie_ID')
movie_info = movie_info.drop('genres_y', axis = 1)
movie_info = movie_info.drop('Freebase_movie_ID_y', axis = 1)
movie_info = movie_info.rename(columns={'genres_x': 'genres', 'Freebase_movie_ID_x': 'Freebase_movie_ID'})

In [274]:
print(movie_info.shape)
display(movie_info.head())

(1195557, 31)


,wiki_movie_ID,Freebase_movie_ID,movie_name,date,revenue,runtime,languages,countries,genres,tconst,...,character_name,actor_birth,actor_gender,actor_height,actor_ethnicity,actor_name,actor_age,character_actor_map_ID,Freebase_character_ID,Freebase_actor_ID
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,English Language,United States of America,Thriller,tt0228333,...,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,English Language,United States of America,Thriller,tt0228333,...,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,English Language,United States of America,Thriller,tt0228333,...,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,English Language,United States of America,Thriller,tt0228333,...,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,English Language,United States of America,Thriller,tt0228333,...,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg


I want to clean the data from the columns 'languages', 'countries', 'genres'

In [261]:
movie_info['languages'] = movie_info['languages'].apply(lambda x: x.split('"')[3] if len(x)>3 else np.nan)
movie_info['countries'] = movie_info['countries'].apply(lambda x: x.split('"')[3] if len(x)>3 else np.nan)
movie_info['genres'] = movie_info['genres'].apply(lambda x: x.split('"')[3] if len(x)>3 else np.nan)
print(movie_info.shape)
display(movie_info.head())

In [277]:
movie_info.corr()

,wiki_movie_ID,revenue,runtime,averageRating,numVotes,actor_height,actor_age
wiki_movie_ID,1.000000,-0.041225,0.000214,-0.058764,-0.049165,-0.002249,0.006333
revenue,-0.041225,1.000000,0.194391,0.011683,0.177283,0.001289,0.042098
runtime,0.000214,0.194391,1.000000,0.001034,-0.001481,0.000309,0.002444
averageRating,-0.058764,0.011683,0.001034,1.000000,0.034441,-0.003769,0.000647
numVotes,-0.049165,0.177283,-0.001481,0.034441,1.000000,0.000503,0.010041
actor_height,-0.002249,0.001289,0.000309,-0.003769,0.000503,1.000000,0.005435
actor_age,0.006333,0.042098,0.002444,0.000647,0.010041,0.005435,1.000000


In [279]:
movie['languages'] = movie['languages'].apply(lambda x: x.split('"')[3] if len(x)>3 else np.nan)
movie['countries'] = movie['countries'].apply(lambda x: x.split('"')[3] if len(x)>3 else np.nan)
movie['genres'] = movie['genres'].apply(lambda x: x.split('"')[3] if len(x)>3 else np.nan)
print(movie.shape)
display(movie.head())

(81741, 9)


,wiki_movie_ID,Freebase_movie_ID,movie_name,date,revenue,runtime,languages,countries,genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,English Language,United States of America,Thriller
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,English Language,United States of America,Mystery
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,Norwegian Language,Norway,Crime Fiction
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,English Language,United Kingdom,Thriller
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,German Language,Germany,Drama


In [345]:
movie['month_day'] = movie['date'].apply(lambda x: x[5:] if len(str(x))>3 else np.nan)
movie['year'] = movie['date'].apply(lambda x: x[0:4] if len(str(x))>3 else x)
movie.drop('date', axis = 1)
movie_info['month_day'] = movie_info['date'].apply(lambda x: x[5:] if len(str(x))>3 else np.nan)
movie_info['year'] = movie_info['date'].apply(lambda x: x[0:4] if len(str(x))>3 else ())
movie_info.drop('date', axis = 1)

,wiki_movie_ID,Freebase_movie_ID,movie_name,revenue,runtime,languages,countries,genres,tconst,titleType,...,actor_gender,actor_height,actor_ethnicity,actor_name,actor_age,character_actor_map_ID,Freebase_character_ID,Freebase_actor_ID,month_day,year
0,975900,/m/03vyhn,Ghosts of Mars,14010832.0,98.0,English Language,United States of America,Thriller,tt0228333,movie,...,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7,08-24,2001
1,975900,/m/03vyhn,Ghosts of Mars,14010832.0,98.0,English Language,United States of America,Thriller,tt0228333,movie,...,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4,08-24,2001
2,975900,/m/03vyhn,Ghosts of Mars,14010832.0,98.0,English Language,United States of America,Thriller,tt0228333,movie,...,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l,08-24,2001
3,975900,/m/03vyhn,Ghosts of Mars,14010832.0,98.0,English Language,United States of America,Thriller,tt0228333,movie,...,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc,08-24,2001
4,975900,/m/03vyhn,Ghosts of Mars,14010832.0,98.0,English Language,United States of America,Thriller,tt0228333,movie,...,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg,08-24,2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195552,12476867,/m/02w7zz8,Spliced,NaN,86.0,English Language,Canada,Thriller,tt1596561,tvSeries,...,NaN,NaN,NaN,Billy Morton,NaN,/m/0gchkcy,NaN,/m/0gc4lfm,,2002
1195553,12476867,/m/02w7zz8,Spliced,NaN,86.0,English Language,Canada,Thriller,tt1596561,tvSeries,...,NaN,NaN,NaN,Andrea Runge,19.0,/m/0gckh4f,NaN,/m/0gbx_rk,,2002
1195554,12476867,/m/02w7zz8,Spliced,NaN,86.0,English Language,Canada,Thriller,tt1596561,tvSeries,...,F,NaN,NaN,Wendy Anderson,NaN,/m/0gcp8fv,NaN,/m/0gby01h,,2002
1195555,12476867,/m/02w7zz8,Spliced,NaN,86.0,English Language,Canada,Thriller,tt1596561,tvSeries,...,NaN,NaN,NaN,Ariel Bastian,NaN,/m/0gdkb51,NaN,/m/0gdkb55,,2002


In [357]:
movie[movie.revenue.notna()]

,wiki_movie_ID,Freebase_movie_ID,movie_name,date,revenue,runtime,languages,countries,genres,month_day,year
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,English Language,United States of America,Thriller,08-24,2001
7,10408933,/m/02qc0j7,Alexander's Ragtime Band,1938-08-16,3600000.0,106.0,English Language,United States of America,Musical,08-16,1938
13,171005,/m/016ywb,Henry V,1989-11-08,10161099.0,137.0,English Language,United Kingdom,Costume drama,11-08,1989
17,77856,/m/0kcn7,Mary Poppins,1964-08-27,102272727.0,139.0,English Language,United States of America,Children's/Family,08-27,1964
21,612710,/m/02wjqm,New Rose Hotel,1999-10-01,21521.0,92.0,English Language,United States of America,Thriller,10-01,1999
...,...,...,...,...,...,...,...,...,...,...,...
81695,54540,/m/0f7hw,Coming to America,1988-06-29,288752301.0,117.0,English Language,United States of America,Romantic comedy,06-29,1988
81720,7761830,/m/0kvgqb,Spaced Invaders,1990,15369573.0,100.0,English Language,United States of America,Alien Film,,1990
81725,1918494,/m/0660qx,State and Main,2000-08-26,6944471.0,106.0,Italian Language,France,Parody,08-26,2000
81726,664006,/m/030xw6,Guilty as Sin,1993-06-04,22886222.0,107.0,NaN,United States of America,Thriller,06-04,1993


In [361]:
movie[movie.year == '1010']

,wiki_movie_ID,Freebase_movie_ID,movie_name,date,revenue,runtime,languages,countries,genres,month_day,year
62836,29666067,/m/0fphzrf,Hunting Season,1010-12-02,12160978.0,140.0,Turkish Language,Turkey,Crime Fiction,12-02,1010


In [219]:
unique_movies.head()

,wiki_movie_ID,Freebase_movie_ID,movie_name,date,revenue,runtime,languages,countries,genres,tconst,...,character_name,actor_birth,actor_gender,actor_height,actor_ethnicity,actor_name,actor_age,character_actor_map_ID,Freebase_character_ID,Freebase_actor_ID
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",tt0228333,...,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",tt0228333,...,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",tt0228333,...,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",tt0228333,...,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",tt0228333,...,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg


I will count how many revenues values there are

In [214]:
print('The precentage of null values in the revenues columns is', pd.isnull(movie_info.revenue).sum()*100/movie_info.shape[0])

The precentage of null values in the revenues columns is 58.357067040718256


Therefore it's important to take some other indicator of success rather than only the revenue. I will chose the ratings indicator.

In [73]:
title = title.merge(rating, on='tconst')
title.head()

,wiki_movie_ID,Freebase_movie_ID,movie_name,date,revenue,runtime,languages,countries,genres_x,tconst,...,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres_y,summary,averageRating,numVotes
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",tt0228333,...,Ghosts of Mars,Ghosts of Mars,0,2001,\N,98,"Action,Horror,Sci-Fi","Set in the second half of the 22nd century, th...",4.9,55249
1,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",tt0594845,...,Ghosts of Mars,Ghosts of Mars,0,2001,\N,\N,"Documentary,Short","Set in the second half of the 22nd century, th...",7.9,19
2,23070193,/m/064mmt5,The Gangsters,1956,NaN,NaN,"{""/m/064_8sq"": ""French Language""}","{""/m/0f8l9c"": ""France""}","{""/m/05p553"": ""Comedy film""}",tt0002894,...,The Gangsters,The Gangsters,0,1913,\N,35,"Comedy,Short",This is the story of racketeer Amédée who reca...,6.8,16
3,18998739,/m/04jcqvw,The Sorcerer's Apprentice,2002,NaN,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/0hzlz"": ""South Africa""}","{""/m/0hqxf"": ""Family Film"", ""/m/01hmnh"": ""Fant...",tt0048644,...,The Sorcerer's Apprentice,The Sorcerer's Apprentice,0,1955,\N,13,Short,"Every hundred years, the evil Morgana returns...",6.5,160
4,12621957,/m/05pdd86,The Sorcerer's Apprentice,2010-07-08,215283742.0,111.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06n90"": ""Science Fiction"", ""/m/03k9fj"": ""...",tt0048644,...,The Sorcerer's Apprentice,The Sorcerer's Apprentice,0,1955,\N,13,Short,"In AD 740, one of Merlin's three apprentices...",6.5,160


In [140]:
title = title.merge(character, on = 'Freebase_movie_ID')

No repeated column names in the merged dataset

In [162]:
clean_title = title.drop('wiki_movie_ID_y', axis = 1)
clean_title = clean_title.drop('genres_y', axis = 1)
clean_title = clean_title.rename(columns={"wiki_movie_ID_x": "wiki_movie_ID", 'genres_x': 'genres'})
for col in clean_title.columns:
    print(col)

wiki_movie_ID
Freebase_movie_ID
movie_name
date
revenue
runtime
languages
countries
genres
tconst
titleType
primaryTitle
originalTitle
isAdult
startYear
endYear
runtimeMinutes
summary
averageRating
numVotes
mult
release_date
character_name
actor_birth
actor_gender
actor_height
actor_ethnicity
actor_name
actor_age
character_actor_map_ID
Freebase_character_ID
Freebase_actor_ID


In [165]:
no_id = clean_title.drop('wiki_movie_ID', axis = 1)
no_id = no_id.drop('Freebase_movie_ID', axis = 1)

In [166]:
movie_info[]

movie_name
date
revenue
runtime
languages
countries
genres
tconst
titleType
primaryTitle
originalTitle
isAdult
startYear
endYear
runtimeMinutes
summary
averageRating
numVotes
mult
release_date
character_name
actor_birth
actor_gender
actor_height
actor_ethnicity
actor_name
actor_age
character_actor_map_ID
Freebase_character_ID
Freebase_actor_ID


We can do some describtive statistics of the features in the dataset.

In [177]:
np.array([no_id.movie_name == 'Ghosts of Mars']).sum()

34

We can now see that the columns of average ratings is full and therefore use this two estimators for the success of a movie. Additionally, we will create and 

In [138]:
title[['mult', 'revenue']].corr()

,mult,revenue
mult,1.000000,0.167319
revenue,0.167319,1.000000
